# **NIVEL 1 - Análisis de Ocupación**

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files
path = '/kaggle/input/uab-the-hack-2024/'
path_out = '/kaggle/working/'

## Limpieza y preprocesamiento de datos

Para llevar a cabo un análisis de ocupación preciso, es esencial trabajar con datos limpios y consistentes. Nuestro conjunto de datos está dividido en cinco archivos principales: `caracteristicas.csv`, `recursos_caracteristicas.csv`, `ubicaciones.csv`, `grupos.csv`, y `calendario_grupos.csv`. Cada dataset contiene información crítica sobre las características, recursos, ubicaciones y horarios de los diferentes grupos.

La limpieza de datos implica revisar y transformar estos archivos para asegurar su consistencia, detectar y manejar valores nulos, corregir posibles errores en los datos y estandarizar formatos. Este proceso permitirá obtener una base de datos confiable y optimizada para la fase de análisis, minimizando errores y mejorando la precisión de los resultados. Utilizaremos bibliotecas de Python como `pandas` y `numpy` para realizar esta tarea de manera eficiente y estructurada.

### 1. CARACTERISTICAS DATASET

In [ ]:
carac = pd.read_csv(path + 'caracteristicas.csv', sep=',', low_memory=False)

# Count lines
print("Count lines: ", carac.shape[0])

# Ordenar por 'ID_CARACTERISTICA' para organizar los datos
carac = carac.sort_values(by='ID_CARACTERISTICA')
carac.head()

Para comenzar el análisis de ocupación, es importante asegurarse de que los datos de caracteristicas.csv sean consistentes y libres de valores que puedan introducir errores. Este proceso de limpieza incluye:

1. Ordenar los Datos: Ordenamos por la columna ID_CARACTERISTICA para mejorar la organización y facilitar el acceso a los registros.
2. Reemplazo de Valores Inconsistentes: La columna QL_MAGNITUD_CARACTERISTICA contiene valores similares en diferentes formatos (s/n, S/N, s/n.). Estos fueron reemplazados por un único valor estándar (S/N), mejorando la coherencia de los datos.
3. Eliminación de Filas con Valores No Deseados: 
    * Filas con ID_TP_VAL_CARAC igual a -1 y con ID_CARACTERISTICA igual a -1 fueron eliminadas, ya que estos valores son inadecuados o indicativos de datos faltantes.
    * Filas en la columna ID_SN_BAIXA_TCARAC que contienen el valor S, que indica elementos dados de baja, también fueron excluidas.
    * La columna ID_DATA_BAIXA_TCARAC fue eliminada tras filtrar filas que contenían datos en ella, ya que estos registros son irrelevantes para el análisis.
4. Eliminación de Columnas Irrelevantes: Se eliminan las columnas QL_MAGNITUD_CARACTERISTICA, ID_SN_DUPLICAR_RECURS, ID_SN_BAIXA_TCARAC, y ID_DATA_BAIXA_TCARAC, ya que no aportan información adicional para este análisis.

In [ ]:
# Estandarización de valores en 'QL_MAGNITUD_CARACTERISTICA': reemplazar variantes de 'S/N' por un valor consistente
carac['QL_MAGNITUD_CARACTERISTICA'] = carac['QL_MAGNITUD_CARACTERISTICA'].replace(['s/n', 's/n.'], 'S/N')

# Filtrar filas con valores no deseados
# Eliminar filas con -1 en 'ID_TP_VAL_CARAC' o 'ID_CARACTERISTICA'
carac = carac[(carac['ID_TP_VAL_CARAC'] != -1) & (carac['ID_CARACTERISTICA'] != -1)]

# Eliminar filas marcadas como bajas (valor 'S' en 'ID_SN_BAIXA_TCARAC')
carac = carac[carac['ID_SN_BAIXA_TCARAC'] != 'S']

# Eliminar columnas irrelevantes
carac = carac.drop(columns=['QL_MAGNITUD_CARACTERISTICA', 'ID_SN_DUPLICAR_RECURS', 'ID_SN_BAIXA_TCARAC'])

# Filtrar filas con valores nulos en 'ID_DATA_BAIXA_TCARAC' y eliminar la columna
carac = carac[carac['ID_DATA_BAIXA_TCARAC'].isnull()]
carac = carac.drop(columns=['ID_DATA_BAIXA_TCARAC'])

# Mostrar valores únicos y conteos para cada columna
for col in carac.columns:
    #print(f"* {col} : {carac[col].unique()}")
    print(col)
    #print(carac[col].value_counts())
    carac[col].value_counts().plot(kind='bar', title=f"Frecuencia de valores en {col}", figsize=(20, 10))
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.show()

In [ ]:
# Vista previa del dataset limpio
carac.head()

# Guardar dataset del output
carac.to_csv(path_out + '/caracteristicas.csv', index=False)

La salida incluye gráficos de barras para cada columna, mostrando la frecuencia de valores únicos. Aquí algunos puntos clave observados:

* ID_CARACTERISTICA: Se identificaron 76 valores únicos, cada uno representando una característica específica. La distribución es equilibrada, ya que cada ID_CARACTERISTICA aparece una vez, indicando un conjunto diverso de características.
* DS_CARACTERISTICA: Esta columna contiene descripciones en catalán para cada característica. Al examinar los valores únicos, observamos que hay una amplia variedad de recursos documentados, desde Impressora matricial hasta Capacitat del grup.
* QL_MEMO_CARACTERISTICA: Valores únicos que actúan como identificadores de características específicas (como Codi divisió, Canó, etc.). La distribución aquí también es uniforme, lo que muestra que cada característica es única y específica.
* ID_TP_VAL_CARAC: Los valores más frecuentes son B (55 ocurrencias), seguido de E, L, y C. Esto puede indicar tipos o clasificaciones de características dentro de la base de datos.

### 2. RECURSOS CARACTERISTICAS DATASET

In [ ]:
rec_carac = pd.read_csv(path + 'recursos_caracteristicas.csv', sep=',', low_memory=False)

# Count lines
print("Count lines: ", rec_carac.shape[0])

# Ordenar el DataFrame por la columna ID_RECURS
rec_carac = rec_carac.sort_values(by='ID_RECURS')
rec_carac.head()

El dataset recursos_caracteristicas.csv, es esencial garantizar que los datos sean consistentes y estén libres de valores que puedan introducir errores. Los pasos de limpieza de datos incluyen:

1. Ordenación de los Datos: Los datos se ordenaron por la columna ID_RECURS, mejorando la organización y facilitando el acceso a los registros para el análisis.
2. Eliminación de Columnas Irrelevantes: Se eliminaron columnas como ID_DATA_VIG_INI_VALOR, ID_DATA_VIG_FIN_VALOR, IND_VALOR_CARAC_ALFANUMERIC, IND_VALOR_CARAC_NUMERIC, e IND_PCT_REC_CENTRE. Estas columnas no aportan información relevante para nuestro objetivo actual y su eliminación simplifica el dataset, permitiendo un análisis más preciso y eficiente.
3. Filtrado de Filas con Valores No Deseados: Se eliminaron las filas que contenían el valor -1 en cualquiera de sus columnas. Este valor puede indicar datos no válidos o ausentes y su presencia podría distorsionar el análisis.
4. Manejo de Datos Faltantes: Después de filtrar los datos no deseados, se eliminaron las filas con valores faltantes (NaN). Esta limpieza adicional asegura que el análisis solo utilice datos completos y precisos.
5. Exploración y Visualización de Frecuencia de Valores: Para cada columna en el dataset, se realizó un análisis de los valores únicos y su frecuencia. Se generaron gráficos de barras para visualizar la distribución de los datos en cada columna, lo cual permite identificar rápidamente cualquier irregularidad o patrón en los datos.


In [ ]:
# Eliminar columnas innecesarias
rec_carac = rec_carac.drop(columns=[
    'ID_DATA_VIG_INI_VALOR', 
    'ID_DATA_VIG_FIN_VALOR', 
    'IND_VALOR_CARAC_ALFANUMERIC', 
    'IND_VALOR_CARAC_NUMERIC', 
    'IND_PCT_REC_CENTRE'
])

# Filtrar filas con valores -1
rec_carac = rec_carac[rec_carac != -1].dropna()

# Imprimir los valores únicos y las frecuencias de cada columna
for col in rec_carac:
    #print(f"* {col}: {rec_carac[col].unique()}")
    print(col)
    #print(rec_carac[col].value_counts())
    
    # Graficar las frecuencias de los valores
    rec_carac[col].value_counts().plot(kind='bar', figsize=(20, 10))
    plt.show()

rec_carac.head()


In [ ]:
# Vista previa del dataset limpio
rec_carac.head()

# Guardar dataset del output
rec_carac.to_csv(path_out + '/recursos_caracteristicas.csv', index=False)

En esta etapa, se realiza una verificación final para asegurar la calidad de los datos antes de integrarlos. Primero, se examinan los valores NaN en cada columna de los datasets carac y rec_carac para confirmar que no existan valores faltantes que puedan afectar el análisis. Posteriormente, ambos datasets se combinan utilizando la columna ID_CARACTERISTICA, generando un único dataset que contiene toda la información relevante sobre características y recursos. Esta integración facilitará el análisis posterior y garantizará que se cuente con un conjunto de datos completo y limpio.

In [ ]:
# Verificar valores NaN en cada columna de ambos datasets para asegurar una correcta limpieza de datos
print("Valores nulos en carac:\n", carac.isnull().sum())
print("Valores nulos en rec_carac:\n", rec_carac.isnull().sum())

# Unir los datasets carac y rec_carac usando la columna ID_CARACTERISTICA como clave
merged_data = pd.merge(carac, rec_carac, on='ID_CARACTERISTICA', how='inner')

# Crear la ruta de guardado del dataset combinado
output_path = os.path.join(path_out, 'merged_caracteristicas_recursos.csv')

# Guardar el dataset combinado en un archivo CSV
merged_data.to_csv(output_path, index=False)
print(f"Dataset combinado guardado en: {output_path}")

En este apartado, se analiza el conjunto de datos combinado en el archivo merged_caracteristicas_recursos.csv, el cual contiene información relevante sobre características y recursos. Primero, se carga el dataset y se contabilizan el número total de registros. A continuación, se filtran los datos para incluir únicamente aquellos correspondientes al *ID_CENTRE_ADMINISTRADOR 115*, que representa a la *Escuela de Ingeniería*.

Luego, se realiza un conteo de las ocurrencias de cada característica, lo que permite identificar las más y menos comunes dentro de este contexto específico. Para facilitar la interpretación de los resultados, se generan gráficos de barras que visualizan la distribución de las características en el Centro 115. Esta etapa es crucial para comprender la información y orientar análisis futuros.

In [ ]:
# Cargar los datos combinados
merged_data = pd.read_csv(path_out + 'merged_caracteristicas_recursos.csv', sep=',')

# Contar el número de líneas en el DataFrame
print("Número total de líneas: ", merged_data.shape[0])

# Filtrar las filas donde ID_CENTRE_ADMINISTRADOR es igual a 115
merged_data = merged_data[merged_data['ID_CENTRE_ADMINISTRADOR'] == 115]

# Contar el número de líneas después de aplicar el filtro
print("Número de líneas en la Escuela de Ingeniería: ", merged_data.shape[0])

# Contar el número de ocurrencias de cada DS_CARACTERISTICA y ordenar por conteo
md = merged_data['DS_CARACTERISTICA'].value_counts().sort_values(ascending=True)

# Imprimir los conteos para verificación
print(md)

# Imprimir el conteo de cada característica
'''for i in md.index:
    print(i, ":", md[i])
    print(merged_data[merged_data['DS_CARACTERISTICA'] == i].head(1))
'''
# Graficar el conteo de valores de DS_CARACTERISTICA
md.plot(kind='bar', figsize=(20, 10))
plt.show()

# Convertir la Serie a un DataFrame y restablecer el índice para incluir DS_CARACTERISTICA
md_df = md.reset_index()
md_df.columns = ['DS_CARACTERISTICA', 'Count']  # Renombrar las columnas

# Crear la ruta de salida para el nuevo archivo CSV
output_path = os.path.join(path_out, 'ee_caracteristicas_recursos.csv')

# Guardar el DataFrame en un archivo CSV
md_df.to_csv(output_path, index=False)
print(f"Dataset de la Escuela de Ingeniería guardado en: {output_path}")


### 3. UBICACIONES DATASET

In [ ]:
# Cargar el dataset de ubicaciones
ubi = pd.read_csv(path + 'ubicaciones.csv', sep=',')
ubi.head()  # Mostrar las primeras filas del dataset


Este código se centra en la limpieza, filtrado y análisis de un conjunto de datos sobre ubicaciones en un campus, con el objetivo de obtener información relevante sobre los espacios disponibles. A continuación se describen las etapas clave del proceso:

1. Limpieza de Datos:
    * Se eliminan las filas con valores nulos del DataFrame para garantizar que las operaciones posteriores se realicen sobre datos completos y válidos.
    * Los nombres de las ubicaciones se normalizan a minúsculas, lo que ayuda a prevenir inconsistencias y facilita la búsqueda y comparación de datos.
2. Eliminación de Columnas Irrelevantes:
    Se elimina la columna QL_PORTA, ya que no es necesaria para el análisis, simplificando así el conjunto de datos.
3. Definición de Términos de Inclusión y Exclusión:
    Se establecen listas de términos que se quieren incluir (por ejemplo, 'lab.', 'laboratori', 'aula', 'seminaris') y otros que se desean excluir (por ejemplo, 'accés', 'despatx', 'magatzem'). Esto permite filtrar las ubicaciones y concentrarse en aquellas de interés.
4. Filtrado de Datos:
    Se utiliza un patrón basado en las listas de inclusión y exclusión para filtrar el DataFrame, quedándose únicamente con las ubicaciones que contienen términos relevantes. Este filtrado asegura que solo se consideren espacios como laboratorios y aulas, mientras que se eliminan oficinas y otros espacios no pertinentes.
5. Consolidación de Filas:
    Se concatenan las filas de los edificios específicos (Q, C y B) en un solo DataFrame, facilitando así un análisis más claro de las ubicaciones dentro de esos edificios.
6. Visualización y Análisis:
    * Se cuenta el número de ubicaciones marcadas como "bajas" (no en uso) y se visualiza esta información a través de un gráfico de barras. Esto proporciona una representación visual clara de cuántas ubicaciones están fuera de uso.
    * A continuación, se filtran las ubicaciones para excluir aquellas marcadas como bajas y se elimina la columna correspondiente, ya que no es necesaria para el análisis posterior.
7. Conteo de Ubicaciones por Edificio:
    * Finalmente, se cuenta la cantidad de ubicaciones por edificio y se genera otro gráfico de barras que muestra la distribución de ubicaciones en los edificios considerados, lo que permite visualizar de forma clara la disponibilidad de espacios en el campus.
Este proceso en su conjunto busca proporcionar un conjunto de datos claro y útil para la toma de decisiones sobre el uso y la gestión de los espacios en el campus, centrándose en la relevancia de las ubicaciones educativas y de trabajo.

In [ ]:
# Limpiar el dataframe eliminando filas con valores nulos
ubi = ubi.dropna(axis=0, how='any')

# Normalizar los nombres de las ubicaciones a minúsculas
ubi['DS_UBICACIO'] = ubi['DS_UBICACIO'].str.lower()

# Eliminar la columna QL_PORTA
ubi.drop(columns=['QL_PORTA'], inplace=True)

# Definir listas de inclusiones y exclusiones
inclusiones = ['lab.', 'laboratori', 'aula', 'seminaris']
exclusiones = ['accés', 'despatx', 'magatzem', 'passadís', 'lavabos', 'labavos', 'investigació']

# Crear patrones para las inclusiones y exclusiones
p_inc = "|".join(inclusiones)
p_excl = "|".join(exclusiones)

# Filtrar el dataframe para incluir solo las ubicaciones relevantes
ubi_fin = ubi[ubi['DS_UBICACIO'].str.contains(p_inc)]
ubi_fin = ubi_fin[~ubi_fin['DS_UBICACIO'].str.contains(p_excl, case=False, na=False)]

# Concatenar filas específicas de edificios
ubi_fin = pd.concat([ubi_fin[ubi_fin['ID_EDIFICI'] == 'Q'], 
                      ubi_fin[ubi_fin['ID_EDIFICI'] == 'C'], 
                      ubi_fin[ubi_fin['ID_EDIFICI'] == 'B']], 
                     ignore_index=True)

# Reiniciar el índice del dataframe final
ubi_fin.reset_index(drop=True, inplace=True)

# Mostrar las primeras filas del dataframe final
print(ubi_fin.head())

# Contar la cantidad de valores en ID_SN_BAIXA
baixa_count = ubi_fin['ID_SN_BAIXA'].value_counts()

# Graficar los conteos de ID_SN_BAIXA
plt.figure(figsize=(10, 5))  # Ajustar el tamaño de la figura
plt.bar(baixa_count.index, baixa_count.values)
plt.ylabel('Cantidad')
plt.xlabel('Valor')
plt.title('Conteo de Baixas por Valor')
plt.xticks(rotation=0)  # Rotar etiquetas en el eje x si es necesario
plt.show()

# Filtrar el dataframe final para excluir valores de baja
ubi_fin = ubi_fin[ubi_fin['ID_SN_BAIXA'] == 'N']

# Eliminar la columna ID_SN_BAIXA
ubi_fin.drop(columns=['ID_SN_BAIXA'], inplace=True)

# Reiniciar el índice del dataframe final después de la eliminación
ubi_fin.reset_index(drop=True, inplace=True)

# Visualizar el dataframe final
#print(ubi_fin.head())

# Graficar el conteo de ubicaciones por edificio
edificio_count = ubi_fin['ID_EDIFICI'].value_counts()

# Crear un gráfico de barras para los edificios
plt.figure(figsize=(10, 5))  # Ajustar el tamaño de la figura
plt.bar(edificio_count.index, edificio_count.values)
plt.ylabel('Cantidad de Ubicaciones')
plt.xlabel('ID del Edificio')
plt.title('Conteo de Ubicaciones por Edificio')
plt.xticks(rotation=0)  # Rotar etiquetas en el eje x si es necesario
plt.show()


In [ ]:
ubicaciones.head()

Este bloque de código tiene como objetivo verificar la calidad de los datos en el dataset de *ubicaciones*. Al imprimir la cantidad de valores nulos en cada columna del dataframe, se garantiza que se identifiquen posibles problemas de limpieza antes de realizar análisis posteriores. Esta verificación es crucial para asegurar que los análisis futuros sean precisos y no se vean afectados por datos faltantes.

In [ ]:
print("Valores nulos en ubicaciones:\n", ubi.isnull().sum())

Se considera que cada estudiante necesita aproximadamente 1.5 m². A continuación se describen los pasos clave del proceso:

1. Definición de Variables:
    * Se extraen dos columnas del DataFrame df_fin: ID_TIPUS_DEPENDENCIA, que indica el tipo de dependencia o aula, y IND_METRES, que representa la superficie del aula en metros cuadrados.
    * Se define superficie_alumno como 1.5 m², que es el espacio que se estima que ocupa un alumno.
2. Estimación de Capacidad de Aulas:
    * Se inicializa una lista vacía llamada capacidad_aulas para almacenar las capacidades calculadas.
    * Se itera sobre cada fila del DataFrame usando un bucle for. Para cada aula, se evalúa el valor de ID_TIPUS_DEPENDENCIA:
        - Si el valor es menor que 10, se calcula la capacidad del aula dividiendo IND_METRES entre superficie_alumno. Este cálculo se redondea hacia abajo usando math.floor() para obtener un número entero de alumnos que pueden ocupar el aula.
        - Si el valor de ID_TIPUS_DEPENDENCIA es 10 o mayor, se asume que la capacidad ya está correctamente registrada, y se agrega directamente a la lista capacidad_aulas.
3. Actualización del DataFrame:
    * Se eliminan las columnas ID_TIPUS_DEPENDENCIA e IND_METRES del DataFrame, ya que ya no son necesarias después de realizar los cálculos.
    * Se añade una nueva columna al DataFrame llamada CAPACIDAD, que contiene los valores estimados o los valores originales de capacidad según corresponda.
4. Exportación del DataFrame:
    Finalmente, el DataFrame df_fin, que ahora incluye la capacidad estimada de las aulas, se guarda en un nuevo archivo CSV en la ruta especificada "ubicaciones_cleaned.csv". Este archivo contiene la información actualizada sobre las ubicaciones, lista para su análisis o uso posterior.

In [ ]:
# Definimos el espacio que ocupa un alumno en metros cuadrados
superficie_alumno = 1.5

# Calculamos la capacidad estimada de las aulas
# Usamos np.where para asignar la capacidad según las condiciones especificadas
ubi_fin['CAPACIDAD'] = np.where(
    ubi_fin['ID_TIPUS_DEPENDENCIA'] < 10,
    np.floor(ubi_fin['IND_METRES'] / superficie_alumno),  # Capacidad estimada
    ubi_fin['ID_TIPUS_DEPENDENCIA']  # Usar la capacidad original
)

# Eliminar columnas que ya no son necesarias
ubi_fin.drop(columns=['ID_TIPUS_DEPENDENCIA', 'IND_METRES'], inplace=True)

# Guardar el DataFrame actualizado en un archivo CSV
output_path = path_out + 'ubicaciones_cleaned.csv'
ubi_fin.to_csv(output_path, index=False)
print(f"Dataset de la Escuela de Ingeniería guardado en: {output_path}")

### 4. GRUPOS DATASET

In [ ]:
# Cargar el dataset de ubicaciones
grupos = pd.read_csv(path + 'grupos.csv', sep=',', low_memory=False)

# Filtrar datos para el año académico 2024
grupos = grupos[grupos['ID_CURSO_ACADEMICO'] == 2024].drop(columns=['ID_SEMESTRE'])

# Count lines
print("Count lines: ", grupos.shape[0])

# Ordenar el DataFrame por la columna ID_RECURS
grupos = grupos.sort_values(by='ID_GRUPO')
grupos.head()

En este apartado, realizamos la limpieza del conjunto de datos correspondiente al año académico 2024. A través de diversos pasos, filtramos, codificamos y eliminamos columnas innecesarias para obtener un dataset más manejable y relevante.

1. Filtrado de Datos: Seleccionamos solo las filas correspondientes al año académico 2024 y eliminamos columnas que no aportan información útil, como ID_SEMESTRE.
2. Codificación de Variables: Transformamos la variable ID_PERIODO_DOCENTE en valores numéricos para simplificar el análisis y facilitar el uso de técnicas estadísticas.
3. Filtrado Adicional: Mantuvimos únicamente los grupos con un número positivo de alumnos y horas previstas, asegurando que nos enfocamos en grupos operativos.

In [ ]:
# Imprimir el número de entradas y semestres únicos
print(f'Número de entradas: {len(grupos)}')
print(f'Número de semestres: {grupos["ID_PERIODO_DOCENTE"].nunique()}')

# Codificar ID_PERIODO_DOCENTE en valores numéricos
grupos['ID_PERIODO_DOCENTE'] = pd.Categorical(grupos['ID_PERIODO_DOCENTE']).codes

# Contar grupos por periodo docente
grupo_counts = grupos.groupby('ID_PERIODO_DOCENTE').size()
print(grupo_counts)

# Comprobar existencia del grupo ID -1
group_id = -1
if group_id in grupos['ID_GRUPO'].values:
    print(f'El grupo ID {group_id} existe en el conjunto de datos.')
else:
    print(f'El grupo ID {group_id} no existe en el conjunto de datos.')

# Eliminar columnas innecesarias
columns_to_drop = [
    "IDIOMA_MAJORITARI", 
    "ID_IDIOMA", 
    "ID_UNIDAD_DOCENTE", 
    "ID_DEPARTAMENT_DELEG_DECODA", 
    "ID_VALOR_PERIODO_DOC", 
    "ID_CENTRO_DEPART"
]
grupos.drop(columns=columns_to_drop, inplace=True)

# Imprimir el número de entradas restantes y los tipos de docencia únicos
print(f'Número de entradas restantes: {len(grupos)}')
#print(grupos['ID_TIPO_DOCENCIA'].value_counts())
print(f'Número de tipos de docencia únicos: {grupos["ID_TIPO_DOCENCIA"].nunique()}')

# Filtrar filas con alumnos y horas previstas mayores que cero
grupos = grupos[(grupos['IND_ALUMNOS_GRUPO_REAL'] > 0) & (grupos['IND_HORAS_PREVISTAS'] > 0)]

# Gráfico de la distribución de alumnos por grupo
plt.figure(figsize=(12, 6))
sns.histplot(grupos['IND_ALUMNOS_GRUPO_REAL'], bins=30, kde=True)
plt.title('Distribución de Alumnos por Grupo')
plt.xlabel('Número de Alumnos')
plt.ylabel('Frecuencia')
plt.grid()
plt.show()

# Gráfico de grupos por tipo de docencia
plt.figure(figsize=(12, 6))
tipo_docencia_counts = grupos['ID_TIPO_DOCENCIA'].value_counts()
sns.barplot(x=tipo_docencia_counts.index, y=tipo_docencia_counts.values)
plt.title('Cantidad de Grupos por Tipo de Docencia')
plt.xlabel('Tipo de Docencia')
plt.ylabel('Número de Grupos')
plt.xticks(rotation=45)
plt.grid()
plt.show()

# Guardar el conjunto de datos limpio en un archivo CSV
grupos.to_csv('grupos_cleaned.csv', index=False)

# Nota sobre grupos eliminados si no tienen entradas en el horario


### 5. CALENDARIO GRUPOS DATASET

In [ ]:
# Cargar el conjunto de datos para su análisis
calendario_grupos = pd.read_csv(path + 'calendario_grupos.csv', sep=',', low_memory=False)
# Mostrar las primeras filas del conjunto de datos para entender su estructura
calendario_grupos.head()

En este apartado, hemos llevado a cabo una serie de pasos para cargar y limpiar el conjunto de datos correspondiente a los grupos en el calendario académico. Primero, hemos importado el archivo CSV y visualizado las primeras filas para entender su estructura. A continuación, filtramos las ubicaciones que contienen la letra 'Q', lo que nos permite enfocarnos en las áreas de interés.

Se eliminan varias columnas que no aportan información relevante al análisis, lo que facilita la manipulación y comprensión de los datos. Además, se filtran los grupos que tienen un ID inválido para asegurar que trabajamos solo con información confiable.

Finalmente, se eliminan los duplicados y se guarda el conjunto de datos limpio en un nuevo archivo CSV.

In [ ]:
# Filtrar el conjunto de datos para incluir solo las ubicaciones que contienen 'Q'
#calendario_grupos = calendario_grupos[calendario_grupos['ID_UBICACION'].str.contains(r'Q', regex=True, na=False)]

# Verificar los valores únicos en la columna 'ID_ORIGEN'
print(calendario_grupos['ID_ORIGEN'].unique())

# Eliminar columnas que no son relevantes para nuestro análisis
columns_to_drop = [
    'ID_ORIGEN', 'IND_HORAS_CALENDARIO', 'IND_PCT_CLAU', 
    'ID_UNITAT_COST', 'ID_PROJECTE', 'ID_CLAU_ACCES', 
    'ID_TIPO_RESERVA', 'ID_CENTRE_COST', 'ID_QUART_HORA', 
    'ID_SESION', 'IND_METRES', 'ID_CURSO_ACADEMICO', 
    'ID_RESPONSABLE_RESERVA', 'ID_UBICACION_SAMAS', 'ID_UBICACION'
]
calendario_grupos = calendario_grupos.drop(columns=columns_to_drop)

# Filtrar grupos que tienen un ID válido
calendario_grupos = calendario_grupos[calendario_grupos['ID_GRUPO'] != "-1"]

# Eliminar duplicados
calendario_grupos = calendario_grupos.drop_duplicates()

# Guardar el conjunto de datos limpio en un archivo CSV
calendario_grupos.to_csv(path_out + 'calendario_grupos_clean.csv', index=False)

# Mostrar el conjunto de datos limpio
print(calendario_grupos.head())

# Crear un gráfico de barras que muestre la distribución de grupos por ID_UBICACION
plt.figure(figsize=(12, 6))
calendario_grupos['ID_GRUPO'].value_counts().plot(kind='bar')
plt.title('Distribución de Grupos')
plt.tight_layout()
plt.show()

print(calendario_grupos.head())

## Identificación de ineficiencias

## Visualización de patrones de ocupación